In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
 #   for filename in filenames:
  #      print(os.path.join(dirname, filename))
os.chdir("/kaggle/input/src-h5")
#############os.listdir()
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import numpy as np
import torch
import torch.nn as nn
from tqdm import tqdm
from torch.nn import functional as F
import random
random.seed(0)
import dataset
import model
import trainer
import utils

In [3]:
#### CausalSelfAttention

In [4]:
import model
import trainer

In [5]:
### pretrain 
device = torch.cuda.current_device() if torch.cuda.is_available() else 'cpu'
block_size = 128
text = open('wiki.txt').read()
pretrain_dataset = dataset.CharCorruptionDataset(text, block_size)
mconf = model.GPTConfig(pretrain_dataset.vocab_size, pretrain_dataset.block_size,
    n_layer=4, n_head=8, n_embd=256,additive=False)
model = model.GPT(mconf)
tconf = trainer.TrainerConfig(max_epochs=650,batch_size=128,learning_rate=6e-3,lr_decay=True\
    ,warmup_tokens=512*20,final_tokens=200*len(pretrain_dataset)*block_size,num_workers=4,ckpt_path="/kaggle/working/model")

trainer = trainer.Trainer(model, pretrain_dataset, None, tconf)
trainer.train()

data has 418352 characters, 256 unique.
number of parameters: 3323392


/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
epoch 1 iter 22: train loss 3.44878. lr 5.999655e-03: 100%|██████████| 23/23 [00:03<00:00,  6.17it/s]
epoch 2 iter 22: train loss 3.10700. lr 5.998582e-03: 100%|██████████| 23/23 [00:02<00:00,  8.10it/s]
epoch 3 iter 22: train loss 2.98457. lr 5.996780e-03: 100%|██████████| 23/23 [00:02<00:00,  8.00it/s]
epoch 4 iter 22: train loss 2.82945. lr 5.994250e-03: 100%|██████████| 23/23 [00:02<00:00,  8.27it/s]
epoch 5 iter 22: train loss 2.77263. lr 5.990993e-03: 100%|██████████| 23/23 [00:02<00:00,  8.17it/s]
epoch 6 iter 22: train loss 2.7343

In [7]:
### finetune
birth_qa_train = open('birth_places_train.tsv').read()
birth_qa_dev  = open('birth_dev.tsv').read()
birth_qa_train = dataset.NameDataset(pretrain_dataset,birth_qa_train)
birth_qa_dev = dataset.NameDataset(pretrain_dataset,birth_qa_dev)

mconf = model.GPTConfig(pretrain_dataset.vocab_size, pretrain_dataset.block_size,
    n_layer=4, n_head=8, n_embd=256,additive=False)
pretrained_model = model.GPT(mconf)
pretrained_model.load_state_dict(torch.load("/kaggle/working/model"))
tconf = trainer.TrainerConfig(max_epochs=10,batch_size=256,learning_rate=6e-4, lr_decay=True,  warmup_tokens=512*20,
                              final_tokens=200*len(pretrain_dataset)*block_size,num_workers=4,\
                              ckpt_path="/kaggle/working/model_qa")

trainer = trainer.Trainer(pretrained_model,birth_qa_train, birth_qa_dev, tconf)
trainer.train()

number of parameters: 3323392


epoch 1 iter 7: train loss 0.83039. lr 5.999844e-04: 100%|██████████| 8/8 [00:01<00:00,  4.45it/s]
epoch 2 iter 7: train loss 0.67370. lr 5.999351e-04: 100%|██████████| 8/8 [00:01<00:00,  4.58it/s]
epoch 3 iter 7: train loss 0.59276. lr 5.998521e-04: 100%|██████████| 8/8 [00:01<00:00,  4.57it/s]
epoch 4 iter 7: train loss 0.54037. lr 5.997352e-04: 100%|██████████| 8/8 [00:01<00:00,  4.49it/s]
epoch 5 iter 7: train loss 0.48945. lr 5.995847e-04: 100%|██████████| 8/8 [00:01<00:00,  4.63it/s]
epoch 6 iter 7: train loss 0.43438. lr 5.994004e-04: 100%|██████████| 8/8 [00:01<00:00,  4.36it/s]
epoch 7 iter 7: train loss 0.39434. lr 5.991823e-04: 100%|██████████| 8/8 [00:01<00:00,  4.53it/s]
epoch 8 iter 7: train loss 0.35313. lr 5.989306e-04: 100%|██████████| 8/8 [00:01<00:00,  4.64it/s]
epoch 9 iter 7: train loss 0.31046. lr 5.986453e-04: 100%|██████████| 8/8 [00:01<00:00,  4.42it/s]
epoch 10 iter 7: train loss 0.27290. lr 5.983263e-04: 100%|██████████| 8/8 [00:01<00:00,  4.60it/s]


In [8]:
mconf = model.GPTConfig(pretrain_dataset.vocab_size, pretrain_dataset.block_size,
    n_layer=4, n_head=8, n_embd=256,additive=False)
qa_model = model.GPT(mconf)
qa_model.load_state_dict(torch.load("/kaggle/working/model_qa"))
correct = 0
total = 0

predictions = []
for line in tqdm(open("birth_dev.tsv")):
    x = line.split('\t')[0]
    x = x + '⁇'
    x = torch.tensor([pretrain_dataset.stoi[s] for s in x], dtype=torch.long)[None,...]#.to(torch.cuda.current_device())
    pred = utils.sample(qa_model, x, 32, sample=False)[0]
    completion = ''.join([pretrain_dataset.itos[int(i)] for i in pred])
    pred = completion.split('⁇')[1]
    predictions.append(pred)

total, correct = utils.evaluate_places("birth_dev.tsv", predictions)
if total > 0:
    print('Correct: {} out of {}: {}%'.format(correct, total, correct/total*100))
else:
    print('Predictions written to {}; no targets provided'.format(args.outputs_path))

number of parameters: 3323392


500it [01:56,  4.31it/s]

Correct: 54.0 out of 500.0: 10.8%
